In [1]:
import ROOT
import numpy as np
import matplotlib.pyplot as plt
import sndisplay as sn
import uproot as uproot
import pandas as pd

Welcome to JupyROOT 6.24/02


In [2]:
from matplotlib import cycler, patches

IPython_default = plt.rcParams.copy()
SMALL_SIZE = 8
MEDIUM_SIZE = 10
BIGGER_SIZE = 12

figsize = (4.5, 3)

plt.rc('font', size=SMALL_SIZE)  # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)  # fontsize of the axes title
plt.rc('axes', labelsize=SMALL_SIZE)  # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)  # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
plt.rcParams["font.family"] = "serif"

marker_size = 3
line_width = 0.5

colors = cycler('color', ['#EE6666', '#3388BB', '#9988DD', '#EECC55', '#88BB44', '#FFBBBB'])
plt.rc('axes', facecolor='white', edgecolor='black',
       axisbelow=True, grid=True, prop_cycle=colors)

In [3]:
input_file_name = "/Users/williamquinn/Desktop/read_red/snemo_run-619_red.root"
file = ROOT.TFile(input_file_name, "READ")
file.ls()
tree = file.RED
tree.Print()

TFile**		/Users/williamquinn/Desktop/read_red/snemo_run-619_red.root	
 TFile*		/Users/williamquinn/Desktop/read_red/snemo_run-619_red.root	
  KEY: TTree	RED;1	SuperNEMO RED data
******************************************************************************
*Tree    :RED       : SuperNEMO RED data                                     *
*Entries :     2202 : Total =        35348919 bytes  File  Size =    6198511 *
*        :          : Tree compression factor =   5.71                       *
******************************************************************************
*Br    0 :run_id    : run_id/I                                               *
*Entries :     2202 : Total  Size=       9369 bytes  File Size  =        153 *
*Baskets :        1 : Basket Size=      32000 bytes  Compression=  58.04     *
*............................................................................*
*Br    1 :event_id  : event_id/I                                             *
*Entries :     2202 : Total  Siz

In [6]:
c_tdc2sec = 0.625E-08
t_tdc2sec = 1.25E-08
tdc2ns = 0.390625
adc2mv = 0.610352
MAX_GG_TIMES = 10

In [25]:
data = {}
for event in tree:
    ################################################################################################################
     #
    #  Access the branches of the root file
    #  To know what is in the TTree either use:
    #               tree.Print()
    #               OR
    #               branches = tree.GetListOfBranches()
    #               print([branch.GetName() for branch in branches])
    #
    ################################################################################################################
    run_id = event.run_id
    event_id = event.event_id
    # event_clock = event.event_clock                                       # int The clock of the events seconds
    # event_ticks = event.event_ticks                                       # Event times - TODO: not yet implemented
    n_calo_hits = int(event.nb_calo_hits)                                   # int Number of calorimeter hits
    # calo_hit_id = list(event.calo_hit_id)                                 # list(int) List of the calo hit IDs
    calo_om_side_id = list(event.calo_om_side_id)                           # list(int) List of OM sides (1: French, 0: Italian)
    calo_om_wall_id = list(event.calo_om_wall_id)                           # list(int) List of OM walls (0,1) see url:https://nemo.lpc-caen.in2p3.fr/wiki/NEMO/SuperNEMO/Calorimeter
    calo_om_column_id = list(event.calo_om_column_id)                       # list(int) List of OM columns
    calo_om_row_id = list(event.calo_om_row_id)                             # list(int) List of OM rows
    # calo_om_ref_id = list(event.calo_om_ref_id)                           # Don't know?
    # calo_clock = list(event.calo_clock)                                   # Don't know?
    calo_ticks = list(event.calo_ticks)                                     # list(int) Calo TDC (6.25ns)
    calo_lto = list(event.calo_lto)                                         # list(bool) Calo low threshold flag
    calo_ht = list(event.calo_ht)                                           # list(bool) Calo high threshold flag
    # calo_fcr = list(event.calo_fcr)                                       # list(int) Dont' know
    # calo_lt_trigger_counter = list(event.calo_lt_trigger_counter)         # Don't know
    # calo_lt_time_counter = list(event.calo_lt_time_counter)               # Don't know
    calo_fwmeas_baseline = list(event.calo_fwmeas_baseline)                 # list(int) Calo baseline LSB: ADC unit/16
    calo_fwmeas_peak_amplitude = list(event.calo_fwmeas_peak_amplitude)     # list(int) Calo amplitude LSB: ADC unit/8
    calo_fwmeas_peak_cell = list(event.calo_fwmeas_peak_cell)               # list(int) Calo peak cell TDC: 0-1023
    calo_fwmeas_charge = list(event.calo_fwmeas_charge)                     # list(int) Calo charge NO IDEA OF UNITS - do not trust this
    calo_fwmeas_rising_cell = list(event.calo_fwmeas_rising_cell)           # list(int) Calo rising cell (right edge) TDC unit/256
    calo_fwmeas_falling_cell = list(event.calo_fwmeas_falling_cell)         # list(int) Calo falling cell (left edge) TDC unit/256

    calo_waveform = [list(event.calo_waveform)[1024 * i:1024 * (i + 1)] for i in range(n_calo_hits)]  # list(list(int)) Calo waveform

    nb_tracker_hits = event.nb_tracker_hits                                 # int Number of tracker cells that have at least one hit
    # tracker_hit_id = list(event.tracker_hit_id)                           # list(int) TR hit ids
    tracker_cell_side_id = list(event.tracker_cell_side_id)                 # list(int) TR side (1: French, 0: Italian)
    tracker_cell_row_id = list(event.tracker_cell_row_id)                   # list(int) TR row (0-112)
    tracker_cell_layer_id = list(event.tracker_cell_layer_id)               # list(int) TR layer (0-8)
    # tracker_clock = list(event.tracker_clock)                             # list(int) Don't know

    tracker_anode_R0_ticks = [list(event.tracker_anode_R0_ticks)[i * MAX_GG_TIMES:MAX_GG_TIMES * (i + 1)] for i in
                              range(nb_tracker_hits)]  # list(int) Anode time LSB: 12.5 ns
    tracker_anode_R1_ticks = [list(event.tracker_anode_R1_ticks)[i * MAX_GG_TIMES:MAX_GG_TIMES * (i + 1)] for i in
                              range(nb_tracker_hits)]  # list(int) Anode: 1st low threshold LSB: 12.5 ns
    tracker_anode_R2_ticks = [list(event.tracker_anode_R2_ticks)[i * MAX_GG_TIMES:MAX_GG_TIMES * (i + 1)] for i in
                              range(nb_tracker_hits)]  # list(int) Anode: 2nd low threshold LSB: 12.5 ns
    tracker_anode_R3_ticks = [list(event.tracker_anode_R3_ticks)[i * MAX_GG_TIMES:MAX_GG_TIMES * (i + 1)] for i in
                              range(nb_tracker_hits)]  # list(int) Anode: 1st high threshold LSB: 12.5 ns
    tracker_anode_R4_ticks = [list(event.tracker_anode_R4_ticks)[i * MAX_GG_TIMES:MAX_GG_TIMES * (i + 1)] for i in
                              range(nb_tracker_hits)]  # list(int) Anode: 2st high threshold LSB: 12.5 ns
    tracker_bottom_cathode_R5_ticks = [list(event.tracker_bottom_cathode_R5_ticks)[i * MAX_GG_TIMES:MAX_GG_TIMES * (i + 1)] for i in
                                       range(nb_tracker_hits)]  # list(int) Cathode bottom LSB: 12.5 ns
    tracker_top_cathode_R6_ticks = [list(event.tracker_top_cathode_R6_ticks)[i * MAX_GG_TIMES:MAX_GG_TIMES * (i + 1)] for i in
                                    range(nb_tracker_hits)]  # list(int) Cathode top LSB: 12.5 ns

    ################################################################################################################
    #  Process data here either store in a container like in the data dictionary
    #  OR process directly here
    ################################################################################################################

    ################
    # Quality Cuts #
    ################
    if n_calo_hits == 0 or nb_tracker_hits == 0 or nb_tracker_hits > 50:
        # Cut out noisy data
        continue

    calo_om_nums = [None for i in range(n_calo_hits)]
    for i_om in range(n_calo_hits):
        if calo_om_row_id[i_om] == -1:
            # GVETO OM
            calo_om_num = 520 + 128 + calo_om_side_id[i_om] * 32 + calo_om_wall_id[i_om] * 16 + calo_om_column_id[i_om]
        elif calo_om_wall_id[i_om] == -1:
            # MAIN WALL OM
            calo_om_num = calo_om_side_id[i_om] * 20 * 13 + calo_om_column_id[i_om] * 13 + calo_om_row_id[i_om]
        else:
            # XWALL OM
            calo_om_num = 520 + calo_om_side_id[i_om] * 64 + calo_om_wall_id[i_om] * 32 + calo_om_column_id[i_om] * 16 + calo_om_row_id[i_om]
        calo_om_nums[i_om] = calo_om_num

        '''print(calo_om_num, om_id_string(calo_om_num), calo_om_side_id[i_om],
                  calo_om_wall_id[i_om], calo_om_column_id[i_om], calo_om_row_id[i_om])'''

    try:
        # Take the first calorimeter time
        calo_time = min(calo_ticks) * c_tdc2sec
    except ValueError:
        # If there are no calo hits (its likely noise) set the calo time to 0
        calo_time = 0

    tracker_cell_nums = [None for i in range(nb_tracker_hits)]
    tracker_ppts = [None for i in range(nb_tracker_hits)]
    tracker_cell_times = [[] for i in range(nb_tracker_hits)]
    tracker_cell_timestamps = [[] for i in range(nb_tracker_hits)]

    for i_cell in range(nb_tracker_hits):
        tracker_cell_num = tracker_cell_side_id[i_cell] * 113 * 9 + tracker_cell_row_id[i_cell] * 9 + tracker_cell_layer_id[i_cell]
        # Note: There are 10 entries for each of the cell R0-6. I have chosen to use the 0th entry
        # >>> If the value is -9223372036854775808 this is the default value
        # >>> It is when there is a value for one RX but not another RY for a specific cell - ignore these values

        # TDC RX
        rs = [tracker_anode_R0_ticks[i_cell][0],
              tracker_anode_R1_ticks[i_cell][0], tracker_anode_R2_ticks[i_cell][0],
              tracker_anode_R3_ticks[i_cell][0], tracker_anode_R4_ticks[i_cell][0],
              tracker_bottom_cathode_R5_ticks[i_cell][0], tracker_top_cathode_R6_ticks[i_cell][0]]
        tracker_cell_timestamps[i_cell] = rs
        # Time (RX - calo hit time) in µs
        ts = []
        for r in rs:
            if r == -9223372036854775808:
                ts.append(None)
            elif r == 0:
                ts.append(None)
            else:
                ts.append((r*t_tdc2sec - calo_time)*1E6)
        tracker_cell_times[i_cell] = ts
        tracker_cell_nums[i_cell] = tracker_cell_num
        try:
            tracker_ppts[i_cell] = ts[5] + ts[6]
        except TypeError:
            pass

    # Fill the dictionary
    data[event_id] = {
        "event": {"time": calo_time},
        "cells": {cell: {
            "rs": tracker_cell_timestamps[index],
            "ts": tracker_cell_times[index],
            "ppts": tracker_ppts[index]
        } for index, cell in enumerate(tracker_cell_nums)},
        "oms": {om: {
            "amplitudes": -1 * calo_fwmeas_peak_amplitude[index] * adc2mv/8,
            "ht": calo_ht[index]
        } for index, om in enumerate(calo_om_nums)}
    }

In [8]:
print(data[88])

{'event': {'time': 33.705941143749996}, 'cells': {1493: {'rs': [2696475306, 2696477610, 2696478012, 2696477702, 2696478097, 2696477608, 2696478012], 'ts': [0.18125000167401595, 28.981250004278536, 34.0062500043814, 30.13124999995398, 35.068750001698845, 28.95625000576274, 34.0062500043814], 'ppts': 62.96250001014414}, 1510: {'rs': [2696475297, 2696477901, 0, 2696477290, 2696478005, 2696477902, -9223372036854775808], 'ts': [0.06875000480022209, 32.61875000504233, None, 24.98125000016671, 33.918749998917974, 32.631250000747514, None], 'ppts': None}, 1518: {'rs': [2696475310, 2696477698, 0, 2696476835, 2696477787, -9223372036854775808, 2696477701], 'ts': [0.2312500058110345, 30.08125000292239, None, 19.293750000315413, 31.193750004376852, None, 30.118750004248795], 'ppts': None}, 1517: {'rs': [2696479334, 2696481626, 0, 2696479895, 2696481712, 2696481620, -9223372036854775808], 'ts': [50.531250003871264, 79.18125000117016, None, 57.54375000321943, 80.25625000129821, 79.10625000562277, Non

In [10]:
input_file_name = "/Users/williamquinn/Desktop/read_red/red_619_output.root"
new_file = ROOT.TFile(input_file_name, "READ")
new_file.ls()
new_tree = new_file.event_tree
new_tree.Print()

TFile**		/Users/williamquinn/Desktop/read_red/red_619_output.root	
 TFile*		/Users/williamquinn/Desktop/read_red/red_619_output.root	
  KEY: TTree	event_tree;1	
******************************************************************************
*Tree    :event_tree:                                                        *
*Entries :     2171 : Total =         6971671 bytes  File  Size =    2962654 *
*        :          : Tree compression factor =   2.35                       *
******************************************************************************
*Br    0 :event_number : event_number/I                                      *
*Entries :     2171 : Total  Size=       9282 bytes  File Size  =       3158 *
*Baskets :        1 : Basket Size=      32000 bytes  Compression=   2.78     *
*............................................................................*
*Br    1 :event_time : event_time/D                                          *
*Entries :     2171 : Total  Size=      17964 byt

In [27]:
new_data = {}
for event in new_tree:
    if len(event.calo_om_num) == 0 or len(event.tracker_time_anode) == 0:
        continue
    if len(event.tracker_time_anode) > 50:
        continue
        
    calo_time = min(list(event.calo_tdc))*c_tdc2sec*1e6
    tracker_cell_nums = []
    tracker_cell_times = []
    tracker_cell_timestamps = []
    tracker_ppts = []
    for i, cell in enumerate(list(event.tracker_cell_num)):
        rs_ = [None, None, None, None, None, None, None]
        ts_ = [None, None, None, None, None, None, None]
        ppt_ = None
        
        r0 = event.tracker_timestamp_r0[i]
        r1 = event.tracker_timestamp_r1[i]
        r2 = event.tracker_timestamp_r2[i]
        r3 = event.tracker_timestamp_r3[i]
        r4 = event.tracker_timestamp_r4[i]
        r5 = event.tracker_timestamp_r5[i]
        r6 = event.tracker_timestamp_r6[i]
        rs = [r0,r1,r2,r3,r4,r5,r6]
        if r0 > 0 and r5 > 0 and r6 > 0:
            t0 = r0*t_tdc2sec*1e6 - calo_time
            t1 = r1*t_tdc2sec*1e6 - calo_time
            t2 = r2*t_tdc2sec*1e6 - calo_time
            t3 = r3*t_tdc2sec*1e6 - calo_time
            t4 = r4*t_tdc2sec*1e6 - calo_time
            t5 = r5*t_tdc2sec*1e6 - calo_time
            t6 = r6*t_tdc2sec*1e6 - calo_time
            ppt = t5+t6
            ts = [t0,t1,t2,t3,t4,t5,t6]
        tracker_cell_nums.append(cell)
        tracker_cell_timestamps.append(rs_)
        tracker_cell_times.append(ts_)
        tracker_ppts.append(ppt_)
            
    new_data[event.event_number] = {
        "event": {"time": calo_time},
        "cells": {cell: {
            "rs": tracker_cell_timestamps[index],
            "ts": tracker_cell_times[index],
            "ppts": tracker_ppts[index]
        } for index, cell in enumerate(tracker_cell_nums)},
        "oms": {om: {
            "amplitudes": -1 * event.calo_amplitude[index] * adc2mv/8,
            "ht": event.calo_high_t[index]
        } for index, om in enumerate(list(event.calo_om_num))}
    }

In [31]:
a = list(data.keys())
b = list(new_data.keys())
for i in data.keys():
    if new_data[88]["cells"].keys() == data[i]["cells"].keys():
        print(i)
        print(data[i]["cells"].keys(), new_data[88]["cells"].keys())
        print(data[i]["oms"])
        print(new_data[88]["oms"])

90
dict_keys([1472, 1484, 1483, 1450, 1473, 1449, 1474, 1475, 1461, 1462]) dict_keys([1472, 1484, 1483, 1450, 1473, 1449, 1474, 1475, 1461, 1462])
{368: {'amplitudes': 33.416772, 'ht': 0}, 369: {'amplitudes': 4.272464, 'ht': 0}, 379: {'amplitudes': 2.746584, 'ht': 0}, 380: {'amplitudes': 215.072786, 'ht': 1}}
{368: {'amplitudes': -67.427802734375, 'ht': 0}, 369: {'amplitudes': -69.8492431640625, 'ht': 0}, 379: {'amplitudes': -69.662978515625, 'ht': 0}, 380: {'amplitudes': -53.03885864257813, 'ht': 1}}


In [38]:
for event in data:
    do_event = False
    calo_tot = 0
    for om in data[event]["oms"].keys():
        ht = data[event]["oms"][om]["ht"]
        calo_tot += ht
    
                
    if calo_tot != 2:
        continue
        
    tr_tot = 0
    for cell in data[event]["cells"].keys():
        t0 = data[event]["cells"][cell]["ts"][0]
        t5 = data[event]["cells"][cell]["ts"][5]
        t6 = data[event]["cells"][cell]["ts"][6]
        if t0 is not None:
            tr_tot += 1
    
    if tr_tot > 5:
        print(event)

97
151
154
188
190
258
384
450
467
588
606
670
678
725
822
826
876
961
1036
1098
1188
1192
1215
1254
1436
1447
1455
1466
1485
1499
1515
1540
1587
1734
1815
1837
1882
1957
1996
2012
2051
2055
2083
2175


In [39]:
data[97]

{'event': {'time': 36.2577565625},
 'cells': {1454: {'rs': [2900620525,
    2900622200,
    2900624227,
    2900622294,
    2900624316,
    2900624228,
    2900622207],
   'ts': [0.0,
    20.937499996875886,
    46.275000002538036,
    22.112499998172552,
    47.38749999688707,
    46.28749999824322,
    21.02500000233931],
   'ppts': 67.31250000058253},
  1455: {'rs': [2900620622,
    2900622231,
    2900624570,
    2900622329,
    2900624666,
    2900624576,
    2900622255],
   'ts': [1.2125000026230737,
    21.324999998739713,
    50.56250000023965,
    22.54999999706797,
    51.762500000052114,
    50.63749999578704,
    21.625000002245542],
   'ppts': 72.26249999803258},
  470: {'rs': [2900620587,
    2900622825,
    2900623936,
    2900622925,
    2900624035,
    2900623943,
    2900622833],
   'ts': [0.7749999966222276,
    28.7499999984675,
    42.63750000177424,
    30.000000002416982,
    43.874999995807684,
    42.72500000013224,
    28.84999999963611],
   'ppts': 71.5749999

In [41]:
event_data = data[97]
sndemo = sn.demonstrator(f"event_{97}")
for om in range(712):
    if om in event_data["oms"].keys():
                sndemo.setomcontent(om, 1)
for cell in range(2034):
    if cell in event_data["cells"].keys():
        t0 = event_data["cells"][int(cell)]["ts"][0]
        t5 = event_data["cells"][int(cell)]["ts"][5]
        t6 = event_data["cells"][int(cell)]["ts"][6]
        if t0 is not None and t5 is not None and t6 is not None:
            sndemo.setggcontent(cell, 1)
try:
    sndemo.draw_top()
    sndemo.save("/Users/williamquinn/Desktop/read_red")
except ValueError:
    print("sndisplay problem")

Info in <TCanvas::Print>: pdf file /Users/williamquinn/Desktop/read_red/event_97_d.pdf has been created
